# 제주도 핫플레이스 - 인스타그램 크롤링

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote

In [2]:
import unicodedata
import re
from tqdm.notebook import tqdm

In [20]:
chromedriver ='/Users/youngseolee/Downloads/chromedriver'
driver = webdriver.Chrome(chromedriver)

### 1. Instagram 에 접속하여 로그인

In [21]:
insta_url = 'https://www.instagram.com'
driver.get(insta_url)
time.sleep(1)

In [22]:
with open('password.txt') as f:
    password= f.read(10)

### 1. 인스타그램 로그인하기

In [23]:
user='seoya_lee'
input_user = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_user.clear()
input_user.send_keys(user)

In [24]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)
input_pwd.submit()
time.sleep(1)

In [25]:
# 로그인 정보 저장 - 나중에하기
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass

In [26]:
# 알림 설정 - 나중에하기
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

In [12]:
# 팝업 화면에서 나가기('x' 클릭)
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/button"}
  (Session info: chrome=92.0.4515.159)


### 6. 여러개의 게시글 가져오기

In [27]:
def get_content(driver):
    html= driver.page_source
    soup= BeautifulSoup(html,'html.parser')
    
    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC',content)
    except:
        content= ' '
        
    
    tags = re.findall(r'#[^\s#,\\]+', content)
    date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]

    try:
        like= soup.select_one('div.Nm9Fw').text[4:-1]
    except:
        like = 0 
    
    try:
        place = soup.select_one('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place= ''
        
    return [content, date, like, place, tags]

### - 제주도 맛집, 제주 맛집, 제주도관광, 제주여행 

In [15]:
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[3]/button"}
  (Session info: chrome=92.0.4515.159)


In [28]:
keyword = '제주맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
driver.get(url)

In [29]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [30]:
results = []
for _ in tqdm(range(200)):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(5)
    except:
        time.sleep(3)
        driver.find_elements_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(5)


  0%|          | 0/200 [00:00<?, ?it/s]

AttributeError: 'list' object has no attribute 'click'

In [20]:
df = pd.DataFrame(results, columns= ['content', 'date', 'like', 'place','tags'])
df.head()

,content,date,like,place,tags
0,.#연동 #모니카옛날통닭간판부터 맛집포스 작렬하는 찐맛집이젠 인생치킨이라고 말하기도...,2021-08-19,"1,013",,"[#연동, #모니카옛날통닭간판부터, #제주맛집, #제주도맛집, #제주공항근처맛집, ..."
1,.빵수니 빵도리들을 위한 <제주 빵지순례 모음! >맛있고 가볼만한 빵집들을 모아봤어...,2021-08-19,"1,046",,"[#제주맛집, #제주도맛집, #제주공항근처맛집, #제주시맛집, #애월맛집, #중문맛..."
2,그 유명한 #연돈 에 다녀왔습니다. 그런데ㅎ;;;⠀⠀⠀⠀⠀⠀⠀어마무시한 웨이팅으로 ...,2021-08-18,"5,150",,"[#연돈, #제주도연돈, #테이블링, #등심까스, #수제볼카츠, #아토짱_제주, #..."
3,42년 전통 도민 맛집. 태양을 품은 듯 화끈한 비주얼이다. 포슬포슬한 두부가 푸짐...,2021-08-18,"1,331",,"[#신해바라기분식, #제주도맛집뜨끈한, #순두부찌개, #국밥맛집, #제주맛집, #제..."
4,.#연동 #모니카옛날통닭간판부터 맛집포스 작렬하는 찐맛집이젠 인생치킨이라고 말하기도...,2021-08-19,"1,261",,"[#연동, #모니카옛날통닭간판부터, #제주맛집, #제주도맛집, #제주공항근처맛집, ..."


In [21]:
df.tail()

,content,date,like,place,tags
195,#빈_오월에제주 #항공샷⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ ⠀⠀⠀...,2021-08-19,20,,"[#빈_오월에제주, #항공샷⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀]"
196,#수르떡볶이 X 리얼클로젯어릴적 추억의 학교앞 분식집 떡볶이맛수르떡볶이 4종 + 치...,2021-08-19,15,,[#수르떡볶이]
197,#수르떡볶이 X 리얼클로젯어릴적 추억의 학교앞 분식집 떡볶이맛수르떡볶이 4종 + 치...,2021-08-19,15,,[#수르떡볶이]
198,- 제주도의 핫한 함덕해변독채형 가족펜션인 게으르게펜션 입니다..바쁜 일상에서 잠시...,2021-08-19,저 좋아요를 눌러보세,,"[#제주도펜션, #제주펜션, #함덕펜션, #함덕해수욕장펜션, #게으르게펜션, #함덕..."
199,#수르떡볶이 X 리얼클로젯어릴적 추억의 학교앞 분식집 떡볶이맛수르떡볶이 4종 + 치...,2021-08-19,13,,[#수르떡볶이]


In [22]:
df.to_csv('data/제주도맛집.csv', index=False)

In [23]:
driver.close()

4가지 CSV파일의 중복을 제거한 후 통합 저장 

In [142]:
jeju_df = pd.DataFrame([])
for keyword in ['제주도맛집','제주맛집','제주도관광','제주여행']:
    df= pd.read_csv(f'data/{keyword}.csv')
    print(df.tail())

                                               content        date like  \
195               사진도찍다말다그냥 처먹처먹 하다가 휴가끝 👋#찬진은진 #진바니먹방  2021-08-19   56   
196  .빨주노초파보 🌈쫀득쫀득 꼬끄에 맛있는 필링🤤✨월정 바다와 함께일 때 더 빛이 나는...  2021-08-19    1   
197  제주 금능 “율” 크로플바삭바삭한 크로플과 아이스아메리카노는 꿀조합💙꼬마손님들은 무...  2021-08-19    0   
198  제주도 올때마다 항상 들리는 곳 ! 타르트로 시작하깅 ~~~일일일타르해야징 ㅎㅎㅎ ...  2021-08-19    3   
199  제주에 가신다면 무조건 동귀리갈칫집 다녀와보시길 추천드려요!!생생정보에 나온 맛집인...  2021-08-19  208   

     place                                               tags  
195    NaN                                ['#찬진은진', '#진바니먹방']  
196    NaN  ['#제주월정리', '#세화카페', '#제주월정리카페', '#서귀포맛집', '#제주...  
197    NaN                                                 []  
198    NaN  ['#제주도카페', '#제주도맛집', '#제주핫플', '#제주공항근처카페', '#j...  
199    NaN  ['#동귀리갈칫집', '#제주맛집', '#제주시맛집', '#제주도맛집', '#노형동...  
                                               content        date  \
155  카라바오 망고라고 기네스 북에올라간 최고 맛있는 망고~~!!인생망고 맛집 드디어 찾...  2021-08-19   
156  ⠀⠀⠀⠀ˏ 𓏧 𓏲 𓏲 𓏲 𓋒 𓏲 𓏲 

In [31]:
driver.close()